In [1]:
import numpy as np
import pandas as pd
from math import floor, ceil
from numpy.linalg import cholesky, inv, solve 
from scipy.linalg import cho_solve
from scipy.stats import wishart, invwishart, gamma
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data, summary_data_from_transaction_data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [2]:
def load_dataset(datafile, parse_dates=None):
    df = pd.read_csv(datafile, delimiter=',', parse_dates=parse_dates)
    return df


In [3]:
g_datafolder = '/development/data'
g_customer_dataset = '{}/olist_customers_dataset.csv'.format(g_datafolder)
g_orders_dataset = '{}/olist_orders_dataset.csv'.format(g_datafolder)
g_payments_dataset = '{}/olist_order_payments_dataset.csv'.format(g_datafolder)
g_orderitems_dataset = '{}/olist_order_items_dataset.csv'.format(g_datafolder)


In [4]:
customer_df = load_dataset(g_customer_dataset)
parse_dates = ['order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
orders_df = load_dataset(g_orders_dataset, parse_dates=parse_dates)
payments_df = load_dataset(g_payments_dataset)
orderitems_df = load_dataset(g_orderitems_dataset)
cust_ord_df = orders_df.set_index('customer_id').join(customer_df.set_index('customer_id'), how="inner").reset_index()
cust_ord_df = cust_ord_df.set_index('order_id').join(orderitems_df.set_index('order_id'), how="inner").reset_index()
cust_ord_df['monetary_value'] = np.round(cust_ord_df['price'] + cust_ord_df['freight_value'], 0)
cust_ord_df['order_date'] = cust_ord_df.order_purchase_timestamp.dt.date
cust_ord_df['cancelled'] = 0.0
cust_ord_df['cancelled'][cust_ord_df.order_status == 'canceled'] = 1.0
customer_id_col='customer_unique_id'
datetime_col='order_date'
monetary_value_col='monetary_value' 
calibration_period_end = datetime(2018,4,1).date()
observation_period_end = cust_ord_df.order_purchase_timestamp.max().date()


In [5]:

def add_datediffcolumn(df, col_dt1, col_dt2, colname):
    df[colname] = np.round((df[col_dt1] - df[col_dt2]).dt.days, 0) * 1.0
    #df[colname] = np.round((df[col_dt1] - df[col_dt2]) / np.timedelta64(1, 'M'), 0)
    df[colname][df[colname].isna()] = 0.0
    return df

def get_calibration_holdout_data(df
                               , customer_id_col='id'
                               , datetime_col='date'
                               , calibration_period_end=None
                               , observation_period_end=None
                               , monetary_value_col='value'
                               , covariates=None):
    start_date = df[datetime_col].min()
    allcols = [customer_id_col, datetime_col, monetary_value_col]
    if covariates is not None:
        allcols = allcols + covariates
    df = df[allcols]
    df['obs'] = 0
    df['obs'][df[datetime_col] >= calibration_period_end] = 1
    df['x'] = 1.0
    sort_cols = [customer_id_col, 'obs', datetime_col]
    agg_map = {'x':'count', monetary_value_col:'sum'}
    if covariates is not None:
        for covariate in covariates:
            agg_map[covariate] = 'sum'
    df = df.groupby([customer_id_col, 'obs', datetime_col]).agg(agg_map).reset_index() #.sort_values(sort_cols).groupby([customer_id_col, 'obs']).cumcount()+1
    df['x'] = df.sort_values(sort_cols).groupby([customer_id_col, 'obs']).cumcount()+1    
    df['x'] = df['x'] - 1.0
    df['x'][df.obs == 1] += 1.0 
    df['first'] = df[datetime_col]       
    df['last'] = df[datetime_col]       
    df[monetary_value_col][(df.obs == 0) & (df.x == 0)] = np.nan
    groupby_cols = [customer_id_col, 'obs']
    all_cols = groupby_cols + [datetime_col, monetary_value_col, 'x', 'first', 'last'] + covariates
    agg_map = {monetary_value_col:'mean', 'x':'max', 'first':'min', 'last':'max' }
    if covariates is not None:
        for covariate in covariates:
            agg_map[covariate] = 'sum'
    df = df.sort_values(sort_cols)[all_cols].groupby(groupby_cols).agg(agg_map).reset_index()
    df['endobs'] = calibration_period_end    
    df['endobs'][df.obs == 1] = observation_period_end
    df = add_datediffcolumn(df, 'last', 'first', 't')
    df = add_datediffcolumn(df, 'endobs', 'first', 'T')
    cols = ['x', 't', 'T', monetary_value_col] + covariates
    cal_df = df[df.obs == 0][[customer_id_col] + cols] 
    cal_df.columns = [customer_id_col] + ['{}_cal'.format(colname) for colname in cols]
    hold_df = df[df.obs == 1][[customer_id_col] + cols] 
    hold_df.columns = [customer_id_col] + ['{}_holdout'.format(colname) for colname in cols]
    df = cal_df.set_index(customer_id_col).join(hold_df.set_index(customer_id_col), how="left").reset_index().fillna(0.0)
    df['T_holdout'] = np.round((observation_period_end - calibration_period_end).days, 0) * 1.0
    return df


In [18]:
df = get_calibration_holdout_data(cust_ord_df
                                , customer_id_col=customer_id_col
                                , datetime_col=datetime_col
                                , calibration_period_end=calibration_period_end
                                , observation_period_end=observation_period_end
                                , monetary_value_col='monetary_value'
                                , covariates=['cancelled'])

## Implementation

In [ ]:
# x ==> number of repeat purchases
# t ==> First purchase to last purchase
# T ==> First purchase to end of observation period


In [26]:
# Setup Regressors (Covariates) for location of 1st-stage prior, i.e. beta = [log(lambda), log(mu)]
def set_regressors(data, covariates=[]):
    data['intercept'] = 1.0
    covariates = ['intercept'] + covariates
    covars = np.matrix(data[covariates])
    K = len(covariates)
    return covariates, covars, K

def get_diag(shape, val):
    d = np.zeros(shape=shape)
    np.fill_diagonal(d, val) 
    return d

def get_map_from_array(x):
    a_map = {}
    count = 0
    for val in x:
        a_map[val] = count
        count += 1
    return a_map

# set hyper priors "log_lambda", "log_mu"
def set_hyperpriors(K):  
    beta_0 = np.zeros(shape=(K, 2))
    A_0 = get_diag(shape=(K, K), val=0.01) # diffuse precision matrix
    # set diffuse hyper-parameters for 2nd-stage prior of gamma_0; follows defaults from rmultireg example
    nu_00 = 3 + K  # 30
    gamma_00 = get_diag(shape=(2, 2), val=nu_00) # diffuse precision matrix
    hyper_prior = {'beta_0': beta_0, 'A_0':A_0, 'nu_00':nu_00, 'gamma_00':gamma_00}
    return hyper_prior

def draw_z(data, level_1, level_1_params_map):
    tx = data['t_cal']
    Tcal = data['T_cal']
    p_lambda = level_1[level_1_params_map['lambda'], ]
    p_mu = level_1[level_1_params_map['mu'], ]
    mu_lam = p_mu + p_lambda
    t_diff = Tcal - tx
    prob = 1 / (1 + (p_mu / mu_lam) * (np.exp(mu_lam * t_diff) - 1))
    z = (np.random.uniform(size=len(prob)) < prob)
    z[z == True] = 1
    z = z.astype(int)
    return list(z.values)

def draw_tau(data, level_1, level_1_params_map):
    N = len(data)
    tx = data['t_cal']
    Tcal = data['T_cal']
    p_lambda = level_1[level_1_params_map['lambda'], ]
    p_mu = level_1[level_1_params_map['mu'], ]

    mu_lam = p_mu + p_lambda
    z = level_1[level_1_params_map['z'], ]

    alive = (z == 1)
    tau = np.zeros(shape=(N))

    # Case: still alive - left truncated exponential distribution -> [T.cal, Inf]
    if (np.sum(alive) > 0):
        tau[alive] = Tcal[alive] + np.random.exponential(scale=1.0/p_mu[alive], size=np.sum(alive))

    # Case: churned - double truncated exponential distribution -> [tx, T.cal]
    if (np.sum(~alive) > 0):
        mu_lam_tx = np.minimum(700, mu_lam[~alive] * tx[~alive])
        mu_lam_Tcal = np.minimum(700, mu_lam[~alive] * Tcal[~alive])
        rand = np.random.uniform(size=np.sum(~alive))        
        tau[~alive] = (-1.0 * np.log((1.0 - rand) * np.exp(-1.0 * mu_lam_tx) + rand * np.exp((-1.0 * mu_lam_Tcal)))) / mu_lam[~alive]

    return tau

def chol2inv(chol):
    return cho_solve((chol, False), np.eye(chol.shape[0])) 

def draw_wishart(df, scale):
    W = wishart.rvs(df, scale)
    IW = inv(W)
    C = cholesky(W).T
    CI = inv(C)
    return W, IW, C, CI

def rmultireg(Y, X, Bbar, A, nu, V):
    # standard multi-variate normal regression update
    # Slide 33 in http://ice.uchicago.edu/2008_presentations/Rossi/ICE_tutorial_2008.pdf
    n = Y.shape[0]
    m = Y.shape[1]
    k = X.shape[1]    

    RA = cholesky(A)
    W = np.concatenate((X, RA), axis=0) 
    Z = np.concatenate((Y, RA*Bbar), axis=0)
    IR = solve(np.triu(cholesky(np.dot(W.T, W)).T), np.eye(k,k)) #trimatu interprets the matrix as upper triangular and makes solve more efficient
    Btilde = np.dot(np.dot(IR, IR.T), np.dot(W.T,Z))
    E = Z - np.dot(W, Btilde)
    S = np.dot(E.T, E)
    W, IW, C, CI = draw_wishart(df=nu+n, scale=chol2inv(cholesky(V+S).T))
    samples = np.random.normal(size=k*m).reshape(k,m)
    B = Btilde + np.dot(IR, np.dot(samples, CI.T))
    return {'beta': B.T, 'gamma':IW}

def draw_level_2(covars, level_1, level_1_params_map, hyper_prior):
    # standard multi-variate normal regression update
    Y = np.log(level_1[[level_1_params_map['lambda'], level_1_params_map['mu']],].T)
    X = covars
    Bbar = hyper_prior['beta_0']
    A = hyper_prior['A_0']
    nu = hyper_prior['nu_00']
    V = hyper_prior['gamma_00']
    
    return rmultireg(Y, X, Bbar, A, nu, V)

def log_post(log_theta, mvmean, x, z, Tcal, tau, inv_gamma):
    log_lambda = log_theta[0,:] 
    log_mu = log_theta[1,:]
    diff_theta = np.subtract(log_theta, mvmean.T)
    diff_lambda = diff_theta[0,:]
    diff_mu = diff_theta[1,:]
    likel = (x * log_lambda) + ((1 - z) * log_mu) - (((z * Tcal) + (1 - z) * tau) * (np.exp(log_lambda) + np.exp(log_mu)))
    prior = -0.5 * ((np.square(diff_lambda) * inv_gamma[0, 0]) + (2 * np.multiply(diff_lambda, diff_mu) * inv_gamma[0, 1]) + (np.square(diff_mu) * inv_gamma[1, 1]))
    post = np.add(likel[0], prior)    
    post[0,log_mu > 5] = np.NINF  # cap !!
    return post

def step(cur_log_theta, cur_post, gamma, N, mvmean, x, z, Tcal, tau, inv_gamma):
    new_log_theta = cur_log_theta + np.vstack((gamma[0, 0] * np.random.standard_t(df=3, size=N), gamma[1, 1] * np.random.standard_t(df=3, size=N)))
    new_log_theta[0,:] = np.maximum(np.minimum(new_log_theta[0,:], 70), -70)
    new_log_theta[1,:] = np.maximum(np.minimum(new_log_theta[1,:], 70), -70)
    new_post = log_post(new_log_theta, mvmean, x, z, Tcal, tau, inv_gamma)
    # accept/reject new proposal
    mhratio = np.exp(new_post - cur_post)
    unif = np.random.uniform(size=N)
    accepted = np.asarray(mhratio > unif)[0]
    cur_log_theta[:,accepted] = new_log_theta[:, accepted]
    cur_post[0,accepted] = new_post[0,accepted]
    return {'cur_log_theta':cur_log_theta, 'cur_post':cur_post}

def draw_level_1(data, covars, level_1, level_1_params_map, level_2):
    # sample (lambda, mu) given (z, tau, beta, gamma)
    N = len(data)
    x = data['x_cal']
    Tcal = data['T_cal']
    z = level_1[level_1_params_map['z'], ]
    tau = level_1[level_1_params_map['tau'], ]
    mvmean = np.dot(covars, level_2['beta'].T)
    gamma = level_2['gamma']
    inv_gamma = inv(gamma)
    
    cur_lambda = level_1[level_1_params_map['lambda'], ]
    cur_mu = level_1[level_1_params_map['mu'], ]

    # current state
    cur_log_theta = np.vstack((np.log(cur_lambda), np.log(cur_mu)))
    cur_post = log_post(cur_log_theta, mvmean, x, z, Tcal, tau, inv_gamma)
    
    iter = 1  # how high do we need to set this? 1/5/10/100?
    for i in range(0, iter):
        draw = step(cur_log_theta, cur_post, gamma, N, mvmean, x, z, Tcal, tau, inv_gamma)
        cur_log_theta = draw['cur_log_theta']
        cur_post = draw['cur_post']

    cur_theta = np.exp(cur_log_theta)

    return {'lambda':cur_theta[0,:], 'mu':cur_theta[1,:]}

def run_single_chain(data, covariates, K, hyper_prior, nsample, nburnin, nskip):
    ## initialize arrays for storing draws ##
    LOG_LAMBDA = 0
    LOG_MU = 1
    nr_of_cust = len(data)
    #nr_of_draws = nburnin + nsample * nskip
    nr_of_draws = nburnin + nsample

    # The 4 is for "lambda", "mu", "tau", "z"
    level_1_params_map = get_map_from_array(['lambda', 'mu', 'tau', 'z'])
    level_1_draws = np.zeros(shape=(nsample, 4, nr_of_cust))

    level_2_draws = np.zeros(shape=(nsample, (2*K)+3))
    nm = ['log_lambda', 'log_mu']
    if (K > 1):
        nm = ['{}_{}'.format(val2, val1) for val1 in covariates for val2 in nm]
    nm.extend(['var_log_lambda', 'cov_log_lambda_log_mu', 'var_log_mu'])
    level_2_params_map = get_map_from_array(nm)
        
    ## initialize parameters ##
    data['t_cal_tmp'] = data['t_cal']
    data['t_cal_tmp'][data.t_cal == 0] = data['T_cal'][data.t_cal == 0] 
    level_1 = level_1_draws[1,]
    x_cal_mean = np.mean(data['x_cal'])
    t_cal_tmp_mean = np.mean(data['t_cal_tmp'])
    level_1[level_1_params_map['lambda'], ] = x_cal_mean/t_cal_tmp_mean
    level_1[level_1_params_map['mu'], ] = 1 / (data['t_cal'] + 0.5 / level_1[level_1_params_map['lambda'], ])
    
    ## run MCMC chain ##
    hyper_prior['beta_0'][0, LOG_LAMBDA] = np.log(np.mean(level_1[level_1_params_map['lambda'], ]))
    hyper_prior['beta_0'][0, LOG_MU] = np.log(np.mean(level_1[level_1_params_map['mu'], ]))
    
    for i in range(0, nr_of_draws):
        # draw individual-level parameters
        level_1[level_1_params_map['z'], ] = draw_z(data, level_1, level_1_params_map)
        level_1[level_1_params_map['tau'], ] = draw_tau(data, level_1, level_1_params_map)

        level_2 = draw_level_2(covars, level_1, level_1_params_map, hyper_prior)
        draw = draw_level_1(data, covars, level_1, level_1_params_map, level_2)
        level_1[level_1_params_map['lambda'], ] = draw["lambda"]
        level_1[level_1_params_map['mu'], ] = draw["mu"]
        
        #nk = int(round((i - nburnin) / nskip))        
        if (i >= nburnin):
            #Store
            idx = i - nburnin
            level_1_draws[idx,:,:] = level_1 # nolint
            level_2_draws[idx,:] = list(np.array(level_2['beta'].T).reshape(-1)) + [level_2['gamma'][0, 0], level_2['gamma'][0, 1], level_2['gamma'][1,1]]
        if (i % 100) == 0:
            print('draw: {}'.format(i))
    coeff_mean = np.mean(level_2_draws, axis=0)
    coeff_stddev = np.std(level_2_draws, axis=0)    
    coeff = {}
    for param in level_2_params_map:
        coeff[param] = {}
        coeff[param]['mean'] = coeff_mean[level_2_params_map[param]]
        coeff[param]['stddev'] = coeff_stddev[level_2_params_map[param]]
    
    return {"level_1":level_1_draws, "level_1_params_map":level_1_params_map
          , "level_2":level_2_draws, "level_2_params_map":level_2_params_map
          , "coeff": coeff}    

####MCMC Functions
def get_correlation(draws):
    l2pmap = draws["level_2_params_map"]
    draw_means = np.mean(draws['level_2'], axis=0)
    corr = draw_means[l2pmap['cov_log_lambda_log_mu']]/(np.sqrt(draw_means[l2pmap['var_log_lambda']]) * np.sqrt(draw_means[l2pmap['var_log_mu']]))
    return corr

def get_nr_of_cust(draws):
    nr_of_cust = draws["level_1"].shape[2]
    return nr_of_cust

def PAlive(draws):
    l1pmap = draws["level_1_params_map"]
    nr_of_cust = get_nr_of_cust(draws)
    p_alive = np.mean(draws["level_1"][:,l1pmap['z'],:], axis=1)
    return p_alive

def draw_left_truncated_gamma(lower, k, lamda):
    pg = gamma.cdf(x=lower, a=k, scale=1.0/(k*lamda))
    rand = np.random.uniform(1, pg, 1)
    qg = gamma.ppf(q=rand, a=k, scale=1.0/(k*lamda))
    return qg

def DrawFutureTransactions(data, draws, sample_size=None):
    nr_of_draws = draws["level_2"].shape[0]
    if sample_size is not None:
            nr_of_draws = sample_size
    nr_of_cust = get_nr_of_cust(draws)
    parameters = draws["level_1_params_map"]
    x_holdout = np.zeros(shape=(nr_of_draws, nr_of_cust))
    t_cal = data['t_cal']
    T_holdout = data['T_holdout']
    T_cal = data['T_cal']

    for i in range(0, nr_of_cust):
        print('...processing customer: {} of {}'.format(i, nr_of_cust))
        Tcal = T_cal[i]
        Tholdout = T_holdout[i]
        tcal = t_cal[i]
        taus = draws['level_1'][:,parameters['tau'],i]
        ks = np.ones(shape=(len(taus)))    
        lamdas = draws['level_1'][:,parameters['lambda'],i]
        if sample_size is not None:
            taus = taus[sample_size]
            ks = ks[sample_size]
            lambdas = lambdas[sample_size]

        alive = taus > Tcal
        # Case: customer alive
        idx = 0
        for alive_val in alive:
            if alive_val:
                # sample itt which is larger than (Tcal-tx)
                itts = draw_left_truncated_gamma(Tcal - tcal, ks[idx], lamdas[idx])
                # sample 'sufficiently' large amount of inter-transaction times
                minT = np.minimum(Tcal + Tholdout - tcal, taus[idx] - tcal)
                #nr_of_itt_draws = int(np.maximum(10, np.round(minT * lamdas[idx])))
                #itts = np.hstack((itts, np.array(gamma.rvs(a=ks[idx], loc=ks[idx]*lamdas[idx], size=nr_of_itt_draws*2))))
                #if (np.sum(itts) < minT):
                #    itts = np.hstack((itts, np.array(gamma.rvs(a=ks[idx], loc=ks[idx]*lamdas[idx], size=nr_of_itt_draws*4))))
                #if (np.sum(itts) < minT):
                #    itts = np.hstack((itts, np.array(gamma.rvs(a=ks[idx], loc=ks[idx]*lamdas[idx], size=nr_of_itt_draws*800))))
                #if (np.sum(itts) < minT):
                #    print("...not enough inter-transaction times sampled! cust: {}, draw: {}, {} < {}".format(i, idx, np.sum(itts), minT))
                x_holdout[idx, i] = np.sum(np.cumsum(itts) < minT)
            idx += 1
        if (np.any(~alive)):
            x_holdout[~alive, i] = 0
    return x_holdout

def PActive(x_holdout_draws):
    nr_of_cust = x_holdout_draws.shape[1]
    p_alive = np.zeros(shape=(nr_of_cust))    
    for i in range(0, nr_of_cust):
        cd = x_holdout_draws[:,i]
        p_alive[i] = np.mean(cd[cd > 0])
    return p_alive



In [24]:
df.head()

,customer_unique_id,x_cal,t_cal,T_cal,monetary_value_cal,cancelled_cal,x_holdout,t_holdout,T_holdout,monetary_value_holdout,cancelled_holdout,intercept,t_cal_tmp
0,0000f46a3911fa3c0805444483337064,0.0,0.0,387.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,387.0
1,0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,171.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,171.0
2,0004aac84e0df4da2b147fca70cf8255,0.0,0.0,138.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,138.0
3,00053a61a98854899e70ed204dd4bafe,0.0,0.0,32.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,32.0
4,0005e1862207bf6ccc02e4228effd9a0,0.0,0.0,393.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,393.0


In [27]:
#parse_dates = ['first']
#g_cbs_dataset = '{}/cbs.csv'.format(g_datafolder)
#df = load_dataset(g_cbs_dataset, parse_dates=parse_dates)
#covariates, covars, K = set_regressors(df, covariates=["cancelled_cal"])
#hyper_prior = set_hyperpriors(K)
#draws = run_single_chain(df, covariates=covariates, K=K, hyper_prior=hyper_prior, nsample=300, nburnin=700, nskip=10)
x_holdout_draws = DrawFutureTransactions(df, draws, sample_size=None)
df['x_predicted'] = np.mean(x_holdout_draws, axis=0)
p_alive = PActive(x_holdout_draws)
df['palive'] = p_alive

# Main routine
#covariates, covars, K = set_regressors(df, covariates=["cancelled_cal"])
#hyper_prior = set_hyperpriors(K)
#draws = run_single_chain(df, covariates=covariates, K=K, hyper_prior=hyper_prior, nsample=30, nburnin=70, nskip=10)


...processing customer: 0 of 63881
...processing customer: 1 of 63881
...processing customer: 2 of 63881
...processing customer: 3 of 63881
...processing customer: 4 of 63881
...processing customer: 5 of 63881
...processing customer: 6 of 63881
...processing customer: 7 of 63881
...processing customer: 8 of 63881
...processing customer: 9 of 63881
...processing customer: 10 of 63881
...processing customer: 11 of 63881
...processing customer: 12 of 63881
...processing customer: 13 of 63881
...processing customer: 14 of 63881
...processing customer: 15 of 63881
...processing customer: 16 of 63881
...processing customer: 17 of 63881
...processing customer: 18 of 63881
...processing customer: 19 of 63881
...processing customer: 20 of 63881
...processing customer: 21 of 63881
...processing customer: 22 of 63881
...processing customer: 23 of 63881
...processing customer: 24 of 63881
...processing customer: 25 of 63881
...processing customer: 26 of 63881
...processing customer: 27 of 63881
..

...processing customer: 225 of 63881
...processing customer: 226 of 63881
...processing customer: 227 of 63881
...processing customer: 228 of 63881
...processing customer: 229 of 63881
...processing customer: 230 of 63881
...processing customer: 231 of 63881
...processing customer: 232 of 63881
...processing customer: 233 of 63881
...processing customer: 234 of 63881
...processing customer: 235 of 63881
...processing customer: 236 of 63881
...processing customer: 237 of 63881
...processing customer: 238 of 63881
...processing customer: 239 of 63881
...processing customer: 240 of 63881
...processing customer: 241 of 63881
...processing customer: 242 of 63881
...processing customer: 243 of 63881
...processing customer: 244 of 63881
...processing customer: 245 of 63881
...processing customer: 246 of 63881
...processing customer: 247 of 63881
...processing customer: 248 of 63881
...processing customer: 249 of 63881
...processing customer: 250 of 63881
...processing customer: 251 of 63881
.

...processing customer: 447 of 63881
...processing customer: 448 of 63881
...processing customer: 449 of 63881
...processing customer: 450 of 63881
...processing customer: 451 of 63881
...processing customer: 452 of 63881
...processing customer: 453 of 63881
...processing customer: 454 of 63881
...processing customer: 455 of 63881
...processing customer: 456 of 63881
...processing customer: 457 of 63881
...processing customer: 458 of 63881
...processing customer: 459 of 63881
...processing customer: 460 of 63881
...processing customer: 461 of 63881
...processing customer: 462 of 63881
...processing customer: 463 of 63881
...processing customer: 464 of 63881
...processing customer: 465 of 63881
...processing customer: 466 of 63881
...processing customer: 467 of 63881
...processing customer: 468 of 63881
...processing customer: 469 of 63881
...processing customer: 470 of 63881
...processing customer: 471 of 63881
...processing customer: 472 of 63881
...processing customer: 473 of 63881
.

...processing customer: 669 of 63881
...processing customer: 670 of 63881
...processing customer: 671 of 63881
...processing customer: 672 of 63881
...processing customer: 673 of 63881
...processing customer: 674 of 63881
...processing customer: 675 of 63881
...processing customer: 676 of 63881
...processing customer: 677 of 63881
...processing customer: 678 of 63881
...processing customer: 679 of 63881
...processing customer: 680 of 63881
...processing customer: 681 of 63881
...processing customer: 682 of 63881
...processing customer: 683 of 63881
...processing customer: 684 of 63881
...processing customer: 685 of 63881
...processing customer: 686 of 63881
...processing customer: 687 of 63881
...processing customer: 688 of 63881
...processing customer: 689 of 63881
...processing customer: 690 of 63881
...processing customer: 691 of 63881
...processing customer: 692 of 63881
...processing customer: 693 of 63881
...processing customer: 694 of 63881
...processing customer: 695 of 63881
.

...processing customer: 891 of 63881
...processing customer: 892 of 63881
...processing customer: 893 of 63881
...processing customer: 894 of 63881
...processing customer: 895 of 63881
...processing customer: 896 of 63881
...processing customer: 897 of 63881
...processing customer: 898 of 63881
...processing customer: 899 of 63881
...processing customer: 900 of 63881
...processing customer: 901 of 63881
...processing customer: 902 of 63881
...processing customer: 903 of 63881
...processing customer: 904 of 63881
...processing customer: 905 of 63881
...processing customer: 906 of 63881
...processing customer: 907 of 63881
...processing customer: 908 of 63881
...processing customer: 909 of 63881
...processing customer: 910 of 63881
...processing customer: 911 of 63881
...processing customer: 912 of 63881
...processing customer: 913 of 63881
...processing customer: 914 of 63881
...processing customer: 915 of 63881
...processing customer: 916 of 63881
...processing customer: 917 of 63881
.

...

...processing customer: 63186 of 63881
...processing customer: 63187 of 63881
...processing customer: 63188 of 63881
...processing customer: 63189 of 63881
...processing customer: 63190 of 63881
...processing customer: 63191 of 63881
...processing customer: 63192 of 63881
...processing customer: 63193 of 63881
...processing customer: 63194 of 63881
...processing customer: 63195 of 63881
...processing customer: 63196 of 63881
...processing customer: 63197 of 63881
...processing customer: 63198 of 63881
...processing customer: 63199 of 63881
...processing customer: 63200 of 63881
...processing customer: 63201 of 63881
...processing customer: 63202 of 63881
...processing customer: 63203 of 63881
...processing customer: 63204 of 63881
...processing customer: 63205 of 63881
...processing customer: 63206 of 63881
...processing customer: 63207 of 63881
...processing customer: 63208 of 63881
...processing customer: 63209 of 63881
...processing customer: 63210 of 63881
...processing customer: 6

...processing customer: 63399 of 63881
...processing customer: 63400 of 63881
...processing customer: 63401 of 63881
...processing customer: 63402 of 63881
...processing customer: 63403 of 63881
...processing customer: 63404 of 63881
...processing customer: 63405 of 63881
...processing customer: 63406 of 63881
...processing customer: 63407 of 63881
...processing customer: 63408 of 63881
...processing customer: 63409 of 63881
...processing customer: 63410 of 63881
...processing customer: 63411 of 63881
...processing customer: 63412 of 63881
...processing customer: 63413 of 63881
...processing customer: 63414 of 63881
...processing customer: 63415 of 63881
...processing customer: 63416 of 63881
...processing customer: 63417 of 63881
...processing customer: 63418 of 63881
...processing customer: 63419 of 63881
...processing customer: 63420 of 63881
...processing customer: 63421 of 63881
...processing customer: 63422 of 63881
...processing customer: 63423 of 63881
...processing customer: 6

...processing customer: 63612 of 63881
...processing customer: 63613 of 63881
...processing customer: 63614 of 63881
...processing customer: 63615 of 63881
...processing customer: 63616 of 63881
...processing customer: 63617 of 63881
...processing customer: 63618 of 63881
...processing customer: 63619 of 63881
...processing customer: 63620 of 63881
...processing customer: 63621 of 63881
...processing customer: 63622 of 63881
...processing customer: 63623 of 63881
...processing customer: 63624 of 63881
...processing customer: 63625 of 63881
...processing customer: 63626 of 63881
...processing customer: 63627 of 63881
...processing customer: 63628 of 63881
...processing customer: 63629 of 63881
...processing customer: 63630 of 63881
...processing customer: 63631 of 63881
...processing customer: 63632 of 63881
...processing customer: 63633 of 63881
...processing customer: 63634 of 63881
...processing customer: 63635 of 63881
...processing customer: 63636 of 63881
...processing customer: 6

...processing customer: 63825 of 63881
...processing customer: 63826 of 63881
...processing customer: 63827 of 63881
...processing customer: 63828 of 63881
...processing customer: 63829 of 63881
...processing customer: 63830 of 63881
...processing customer: 63831 of 63881
...processing customer: 63832 of 63881
...processing customer: 63833 of 63881
...processing customer: 63834 of 63881
...processing customer: 63835 of 63881
...processing customer: 63836 of 63881
...processing customer: 63837 of 63881
...processing customer: 63838 of 63881
...processing customer: 63839 of 63881
...processing customer: 63840 of 63881
...processing customer: 63841 of 63881
...processing customer: 63842 of 63881
...processing customer: 63843 of 63881
...processing customer: 63844 of 63881
...processing customer: 63845 of 63881
...processing customer: 63846 of 63881
...processing customer: 63847 of 63881
...processing customer: 63848 of 63881
...processing customer: 63849 of 63881
...processing customer: 6

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
df.head()

,customer_unique_id,x_cal,t_cal,T_cal,monetary_value_cal,cancelled_cal,x_holdout,t_holdout,T_holdout,monetary_value_holdout,cancelled_holdout,intercept,t_cal_tmp,x_predicted
0,0000f46a3911fa3c0805444483337064,0.0,0.0,387.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,387.0,0.010000
1,0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,171.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,171.0,0.010000
2,0004aac84e0df4da2b147fca70cf8255,0.0,0.0,138.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,138.0,0.003333
3,00053a61a98854899e70ed204dd4bafe,0.0,0.0,32.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,32.0,0.016667
4,0005e1862207bf6ccc02e4228effd9a0,0.0,0.0,393.0,0.0,0.0,0.0,0.0,155.0,0.0,0.0,1.0,393.0,0.020000


In [29]:
df[['x_cal', 'x_holdout', 'x_predicted']].groupby(['x_cal']).mean()

,x_holdout,x_predicted
x_cal,,
0.0,0.009553,0.017241
1.0,0.040109,0.017454
2.0,0.171875,0.018333
3.0,0.571429,0.019524
4.0,0.571429,0.018571
5.0,0.000000,0.013333
6.0,0.000000,0.016667
8.0,7.000000,0.020000
